# COVID Tweets - Database Assignment
#### April 16th, 2021 

##### By Hudson Ukass and Justin Przybliski

Notes: 

In [125]:
import sklearn as sk
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
import altair as alt
import spacy as sp
import matplotlib.pyplot as plt
os.chdir('/Users/hudso/OneDrive/Documents/COMP 4522/COVIDTweets-Machine-Learning')
sp.load('en_core_web_sm')

#Covid Tweets dataset
covidTweets = pd.read_csv("covidtweets.csv")
#Covid Cases Data
#https://www.kaggle.com/gauravduttakiit/covid-19
covidCases = pd.read_csv("worldwide-aggregate.csv")

#Datetime conversions and hour/min/sec rounding 
#for covidTweets and CovidCases df's.
covidTweets["user_created"] = pd.to_datetime(covidTweets["user_created"], errors='coerce')
covidTweets["date"] = pd.to_datetime(covidTweets["date"], errors='coerce')
covidCases["Date"] = pd.to_datetime(covidCases["Date"], errors='coerce')
covidTweets = covidTweets.assign(date=covidTweets.date.dt.round('D'))

#Drop null and irellevant columns about user profiles 
tweetCase = covid.drop(columns=['user_location', 'user_description', 'user_created', 'hashtags', 'source'])

#Merge the covidTweets and covidCases on their dates.
tweetCase = pd.merge(left=tweetCase, right=covidCases, how='left', left_on='date', 
                  right_on='Date')
tweetCase = tweetCase.drop(columns='Date')
tweetCase = tweetCase.sort_values(by='date', ascending=True)
tweetCase.head()

,id,user_name,user_followers,user_friends,user_favourites,user_verified,date,text,retweets,favorites,is_retweet,Confirmed,Recovered,Deaths,Increase rate
43,1337728701820268544,City A.M.,66224,603,771,True,2020-12-12,Trump announces #vaccine rollout 'in less than...,0,2,False,71903752,46933277,1609529,0.890777
44,1337727767551553536,Daily News Egypt,278080,116,765,True,2020-12-12,#FDA authorizes #PfizerBioNTech #coronavirus v...,1,1,False,71903752,46933277,1609529,0.890777
269,1337963149232926723,LatestLY,67096,344,118,True,2020-12-13,COVID-19 Vaccine Update: Initial Doses of Pfiz...,0,1,False,72434583,47328396,1616986,0.738252
276,1337934174443847680,"Nora O'Brien, CEM",4220,4615,7901,False,2020-12-13,#PfizerBioNTech #COVID19vaccines are prepped f...,0,1,False,72434583,47328396,1616986,0.738252
277,1337933177097134082,rao tadepalli,273,66,557,False,2020-12-13,What we can learn from the victory over #Polio...,1,7,False,72434583,47328396,1616986,0.738252


# Machine Learning - Linear Regression
With help of Kaggle Tutorial


In [67]:
#Define the parameters of X and y here. 
covidN = covid.dropna()

#Prediction target
y = covidN.retweets

#Prediction criteria 
covid_features = ['user_friends', 'user_followers', 'user_favourites', 
                    'favorites']
X = covidN[covid_features]
X.head()

,user_friends,user_followers,user_favourites,favorites
0,1692,405,3247,0
2,88,10,155,0
6,5001,2731,69344,4
9,2368,671,20469,0
10,78,1302,339,0


In [90]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
covid_model = DecisionTreeRegressor()
covid_model.fit(X, y)

predicted_retweets = covid_model.predict(X)
mean_absolute_error(y, predicted_retweets)

#In-Sample Scoring
print("Making retweet predictions for the following 5 Tweets:")
print(X.head())
print("The retweet predictions are")
print(covid_model.predict(X.head()))

Making retweet predictions for the following 5 Tweets:
    user_friends  user_followers  user_favourites  favorites
0           1692             405             3247          0
2             88              10              155          0
6           5001            2731            69344          4
9           2368             671            20469          0
10            78            1302              339          0
The retweet predictions are
[0. 0. 0. 0. 0.]


In [87]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

#Out-of-Sample Scoring 
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

covid_model = DecisionTreeRegressor()
covid_model.fit(train_X, train_y)

val_predictions = covid_model.predict(val_X)
#Print the degree of error for out-of-sample
print("General Degree of Error: ", mean_absolute_error(val_y, val_predictions))

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)
# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [5, 50, 500, 5000, 50000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

General Degree of Error:  3.3684726252665356
Max leaf nodes: 5  		 Mean Absolute Error:  4
Max leaf nodes: 50  		 Mean Absolute Error:  3
Max leaf nodes: 500  		 Mean Absolute Error:  3
Max leaf nodes: 5000  		 Mean Absolute Error:  3
Max leaf nodes: 50000  		 Mean Absolute Error:  3


### Optimal number of leaves is: 50

In [91]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, melb_preds))

3.093335650859599


In [107]:
X = covidN.loc[:,['user_friends', 'user_followers', 'user_favourites', 
                    'favorites']].values
y = covidN.retweets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

regressor = RandomForestRegressor(n_estimators=50, random_state=1)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 2.5207186656719527
Mean Squared Error: 1358.2592976472283
Root Mean Squared Error: 36.85456956263671


In [102]:
covidN.retweets.describe()

count    29971.000000
mean         3.868206
std         58.872228
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max       7695.000000
Name: retweets, dtype: float64

In [110]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
regressor = RandomForestRegressor(n_estimators=50, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets